# This notebook is designed for preparing text data for furthur analyses

## First, I read in the original complaints.csv data

In [ ]:
!pip install swifter

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import swifter
import gc
import nltk 
import sklearn 
import collections
import sys
import itertools
import string

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from tqdm.notebook import tqdm


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
complaints = pd.read_csv("/content/drive/MyDrive/QMSS Practicum 2021/complaints.csv")

In [ ]:
complaints.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc. \nis trying to collect...,NaN,TRANSWORLD SYSTEMS INC,FL,335XX,NaN,Consent provided,Web,2019-09-24,Closed with explanation,Yes,NaN,3384392
1,2019-09-19,"Credit reporting, credit repair services, or o...",Credit reporting,Incorrect information on your report,Information belongs to someone else,NaN,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,PA,15206,NaN,Consent not provided,Web,2019-09-20,Closed with non-monetary relief,Yes,NaN,3379500
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,"Over the past 2 weeks, I have been receiving e...",NaN,"Diversified Consultants, Inc.",NC,275XX,NaN,Consent provided,Web,2019-11-08,Closed with explanation,Yes,NaN,3433198
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,Pioneer has committed several federal violatio...,NaN,Pioneer Capital Solutions Inc,CA,925XX,NaN,Consent provided,Web,2019-09-15,Closed with explanation,Yes,NaN,3374555
4,2021-02-02,Vehicle loan or lease,Loan,Struggling to pay your loan,Lender trying to repossess or disable the vehicle,XXXX XXXX employee that works for Regional Acc...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,FL,32792,NaN,Consent provided,Web,2021-02-02,Closed with explanation,Yes,NaN,4111597


In [ ]:
complaints.columns

Index(['Date received', 'Product', 'Sub-product', 'Issue', 'Sub-issue',
       'Consumer complaint narrative', 'Company public response', 'Company',
       'State', 'ZIP code', 'Tags', 'Consumer consent provided?',
       'Submitted via', 'Date sent to company', 'Company response to consumer',
       'Timely response?', 'Consumer disputed?', 'Complaint ID'],
      dtype='object')

## I drop 'Complaint ID', 'ZIP code' and 'Submitted via' since I will not analyze these variales, and I rename 'Consumer complaint narrative' to be 'Consumer_complaint_narrative' for better use.

In [ ]:
complaints.drop(columns=['Complaint ID', 'ZIP code','Submitted via'], inplace=True)

# for col in ['Product', 'Sub-product', 'Issue', 'Sub-issue', 'Company public response', 'State', 'Tags', 'Consumer consent provided?', 'Company response to consumer', 'Timely response?', 'Consumer disputed?']:
#    complaints[col] = complaints[col].astype('category')

complaints = complaints[pd.notnull(complaints['Consumer complaint narrative'])]
complaints=complaints.rename(columns={'Consumer complaint narrative': 'Consumer_complaint_narrative'})
complaints=complaints.rename(columns={'Date received': 'Date_received'})
complaints=complaints.rename(columns={'Company public response': 'Company_public_response'})
complaints=complaints.rename(columns={'Consumer consent provided?': 'Consumer_consent_provided?'})
complaints=complaints.rename(columns={'Date sent to company': 'Date_sent_to_company'})
complaints=complaints.rename(columns={'Company response to consumer': 'Company_response_to_consumer'})
complaints=complaints.rename(columns={'Timely response?': 'Timely_response?'})
complaints=complaints.rename(columns={'Consumer disputed?': 'Consumer_disputed?'})
complaints=complaints.rename(columns={'Sub-product': 'Sub_product'})
complaints=complaints.rename(columns={'Sub_issue': 'Sub_issue'})

complaints.info(verbose=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 667477 entries, 0 to 1968087
Data columns (total 15 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   Date_received                 667477 non-null  object
 1   Product                       667477 non-null  object
 2   Sub_product                   615305 non-null  object
 3   Issue                         667477 non-null  object
 4   Sub-issue                     518055 non-null  object
 5   Consumer_complaint_narrative  667477 non-null  object
 6   Company_public_response       333457 non-null  object
 7   Company                       667477 non-null  object
 8   State                         663908 non-null  object
 9   Tags                          110235 non-null  object
 10  Consumer_consent_provided?    667477 non-null  object
 11  Date_sent_to_company          667477 non-null  object
 12  Company_response_to_consumer  667476 non-null  object
 13

## In the next step, I will do some NLP stuff,

## 1) such as lower case, special character

## 2) remove all the XXXXXX

## I will call fucntions from a NLP utils.

In [ ]:
def clean_text(str_in):
    import re
    tmp = re.sub("[^A-z]+", " ", str(str_in).lower())
    return tmp

In [ ]:
complaints['Consumer_complaint_narrative'] =complaints['Consumer_complaint_narrative'].apply(clean_text)

In [ ]:
complaints.head()

,Date_received,Product,Sub_product,Issue,Sub-issue,Consumer_complaint_narrative,Company_public_response,Company,State,Tags,Consumer_consent_provided?,Date_sent_to_company,Company_response_to_consumer,Timely_response?,Consumer_disputed?
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc is trying to collect a ...,NaN,TRANSWORLD SYSTEMS INC,FL,NaN,Consent provided,2019-09-24,Closed with explanation,Yes,NaN
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,over the past weeks i have been receiving exce...,NaN,"Diversified Consultants, Inc.",NC,NaN,Consent provided,2019-11-08,Closed with explanation,Yes,NaN
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,pioneer has committed several federal violatio...,NaN,Pioneer Capital Solutions Inc,CA,NaN,Consent provided,2019-09-15,Closed with explanation,Yes,NaN
4,2021-02-02,Vehicle loan or lease,Loan,Struggling to pay your loan,Lender trying to repossess or disable the vehicle,xxxx xxxx employee that works for regional acc...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,FL,NaN,Consent provided,2021-02-02,Closed with explanation,Yes,NaN
6,2019-07-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,previously on xx xx xxxx xx xx xxxx and xx xx ...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,NaN,Consent provided,2019-07-26,Closed with explanation,Yes,NaN


In [ ]:
complaints['Consumer_complaint_narrative'] = complaints['Consumer_complaint_narrative'].str.replace("xx+\s","")

In [ ]:
complaints.head()

,Date_received,Product,Sub_product,Issue,Sub-issue,Consumer_complaint_narrative,Company_public_response,Company,State,Tags,Consumer_consent_provided?,Date_sent_to_company,Company_response_to_consumer,Timely_response?,Consumer_disputed?
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc is trying to collect a ...,NaN,TRANSWORLD SYSTEMS INC,FL,NaN,Consent provided,2019-09-24,Closed with explanation,Yes,NaN
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,over the past weeks i have been receiving exce...,NaN,"Diversified Consultants, Inc.",NC,NaN,Consent provided,2019-11-08,Closed with explanation,Yes,NaN
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,pioneer has committed several federal violatio...,NaN,Pioneer Capital Solutions Inc,CA,NaN,Consent provided,2019-09-15,Closed with explanation,Yes,NaN
4,2021-02-02,Vehicle loan or lease,Loan,Struggling to pay your loan,Lender trying to repossess or disable the vehicle,employee that works for regional acceptance ke...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,FL,NaN,Consent provided,2021-02-02,Closed with explanation,Yes,NaN
6,2019-07-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,previously on and i requested that experian se...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,NaN,Consent provided,2019-07-26,Closed with explanation,Yes,NaN


In [ ]:
complaints['Consumer_complaint_narrative'] = complaints['Consumer_complaint_narrative'].astype(str)

In [ ]:
>>> import nltk
>>> nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
complaints['Consumer_complaint_narrative'] = complaints['Consumer_complaint_narrative'].swifter.apply(lambda x: ' '.join([i for i in x.split() if i not in stop]))

In [ ]:
complaints.head()

,Date_received,Product,Sub_product,Issue,Sub-issue,Consumer_complaint_narrative,Company_public_response,Company,State,Tags,Consumer_consent_provided?,Date_sent_to_company,Company_response_to_consumer,Timely_response?,Consumer_disputed?
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc trying collect debt min...,NaN,TRANSWORLD SYSTEMS INC,FL,NaN,Consent provided,2019-09-24,Closed with explanation,Yes,NaN
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,past weeks receiving excessive amounts telepho...,NaN,"Diversified Consultants, Inc.",NC,NaN,Consent provided,2019-11-08,Closed with explanation,Yes,NaN
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,pioneer committed several federal violations p...,NaN,Pioneer Capital Solutions Inc,CA,NaN,Consent provided,2019-09-15,Closed with explanation,Yes,NaN
4,2021-02-02,Vehicle loan or lease,Loan,Struggling to pay your loan,Lender trying to repossess or disable the vehicle,employee works regional acceptance keeps calli...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,FL,NaN,Consent provided,2021-02-02,Closed with explanation,Yes,NaN
6,2019-07-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,previously requested experian send copy verifi...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,NaN,Consent provided,2019-07-26,Closed with explanation,Yes,NaN


In [ ]:
# complaints.to_csv('/content/drive/MyDrive/QMSS Practicum 2021/narratives.csv')

In [ ]:
complaints.dtypes

Date_received                   object
Product                         object
Sub_product                     object
Issue                           object
Sub-issue                       object
Consumer_complaint_narrative    object
Company_public_response         object
Company                         object
State                           object
Tags                            object
Consumer_consent_provided?      object
Date_sent_to_company            object
Company_response_to_consumer    object
Timely_response?                object
Consumer_disputed?              object
dtype: object

In [ ]:
  >>> import nltk
  >>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
complaints['Consumer_complaint_narrative_token']=complaints['Consumer_complaint_narrative'].swifter.apply(word_tokenize)

In [ ]:
complaints.head()

,Date_received,Product,Sub_product,Issue,Sub-issue,Consumer_complaint_narrative,Company_public_response,Company,State,Tags,Consumer_consent_provided?,Date_sent_to_company,Company_response_to_consumer,Timely_response?,Consumer_disputed?,Consumer_complaint_narrative_token
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc trying collect debt min...,NaN,TRANSWORLD SYSTEMS INC,FL,NaN,Consent provided,2019-09-24,Closed with explanation,Yes,NaN,"[transworld, systems, inc, trying, collect, de..."
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,past weeks receiving excessive amounts telepho...,NaN,"Diversified Consultants, Inc.",NC,NaN,Consent provided,2019-11-08,Closed with explanation,Yes,NaN,"[past, weeks, receiving, excessive, amounts, t..."
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,pioneer committed several federal violations p...,NaN,Pioneer Capital Solutions Inc,CA,NaN,Consent provided,2019-09-15,Closed with explanation,Yes,NaN,"[pioneer, committed, several, federal, violati..."
4,2021-02-02,Vehicle loan or lease,Loan,Struggling to pay your loan,Lender trying to repossess or disable the vehicle,employee works regional acceptance keeps calli...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,FL,NaN,Consent provided,2021-02-02,Closed with explanation,Yes,NaN,"[employee, works, regional, acceptance, keeps,..."
6,2019-07-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,previously requested experian send copy verifi...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,NaN,Consent provided,2019-07-26,Closed with explanation,Yes,NaN,"[previously, requested, experian, send, copy, ..."


In [ ]:
complaints['Consumer_complaint_narrative_length'] = complaints['Consumer_complaint_narrative_token'].str.len() 

In [ ]:
complaints.head()

,Date_received,Product,Sub_product,Issue,Sub-issue,Consumer_complaint_narrative,Company_public_response,Company,State,Tags,Consumer_consent_provided?,Date_sent_to_company,Company_response_to_consumer,Timely_response?,Consumer_disputed?,Consumer_complaint_narrative_token,Consumer_complaint_narrative_length
0,2019-09-24,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,transworld systems inc trying collect debt min...,NaN,TRANSWORLD SYSTEMS INC,FL,NaN,Consent provided,2019-09-24,Closed with explanation,Yes,NaN,"[transworld, systems, inc, trying, collect, de...",9
2,2019-11-08,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,past weeks receiving excessive amounts telepho...,NaN,"Diversified Consultants, Inc.",NC,NaN,Consent provided,2019-11-08,Closed with explanation,Yes,NaN,"[past, weeks, receiving, excessive, amounts, t...",29
3,2019-09-15,Debt collection,Other debt,Attempts to collect debt not owed,Debt was result of identity theft,pioneer committed several federal violations p...,NaN,Pioneer Capital Solutions Inc,CA,NaN,Consent provided,2019-09-15,Closed with explanation,Yes,NaN,"[pioneer, committed, several, federal, violati...",72
4,2021-02-02,Vehicle loan or lease,Loan,Struggling to pay your loan,Lender trying to repossess or disable the vehicle,employee works regional acceptance keeps calli...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,FL,NaN,Consent provided,2021-02-02,Closed with explanation,Yes,NaN,"[employee, works, regional, acceptance, keeps,...",24
6,2019-07-26,"Credit reporting, credit repair services, or o...",Credit reporting,Problem with a credit reporting company's inve...,Their investigation did not fix an error on yo...,previously requested experian send copy verifi...,Company has responded to the consumer and the ...,Experian Information Solutions Inc.,CA,NaN,Consent provided,2019-07-26,Closed with explanation,Yes,NaN,"[previously, requested, experian, send, copy, ...",74


## Next, I add the length for each complaints because I want to know whether a longer complaint implies an more angry consumer. (Non-now yet)

In [ ]:
complaints.to_csv('/content/drive/MyDrive/QMSS Practicum 2021/narratives_token.csv')